In [104]:
'''
t_res: number of raw values to include in each power spectrum
bns: number of bins for each power spectrum
cmprs: number of consecutive power spectra to average
'''
t_res = 250
bns   = t_res//4
cmprs = 1

'''
Channel 0 (N1P) - Right Helix
Channel 1 (N2P) - Right ear canal, front facing
Channel 2 (N3P) - Right ear canal, back facing
Channel 3 (N4P) - Left Helix
Channel 4 (N5P) - Left ear canal, front facing
Channel 5 (N6P) - Left ear canal, back facing
Channel 6 (N7P) - Approximately FP1
Channel 7 (N8P) - Right mastoid
'''
#right_ear_nomastoid = [0,1,2]
right_ear = [0,1,2,7]
left_ear = [3,4,5]
fp1 = [6]
#config for which chans to use
channels = fp1
outFile = 'fp1'



In [105]:
import os
import numpy as np
files = os.listdir('data')

def subject_id (filename):
    return filename.split('.')[0]

def task_name (filename):
    return filename.split('.')[1]

subject_ids = list(set(map(subject_id, files)))
task_names = list(set(map(task_name, files)))


def find_with_id (id):
    return lambda filename: subject_id(filename) == id

def find_with_task (task):
    return lambda filename: task_name(filename) == task

# both args are optional
def find_recordings (id=None, task=None):
    lst = files
    if id:
        lst = list(filter(find_with_id(id), lst))
    if task:
        lst = list(filter(find_with_task(task), lst))
    return lst

%pylab inline
import pandas as pd
from os.path import join

def load_recording (filename):
    return pd.read_csv(join('data', filename), header=None)

#load_recording(files[3])


# find_recordings(task='breathe_o')
# find_recordings('002', 'breathe_o')
'''utilities'''
def withold_final (percent, lst):
    n = int(len(lst)*percent)
    return lst[:-1*n, :]
#withold_final(0.4,
#              np.array([[1,0],[2,0],[3,0],[4,0],[5,0]]))
def get_final (percent, lst):
    n = int(len(lst)*percent)
    return lst[-1*n:, :]
#get_final(0.4,
#          np.array([[1,0],[2,0],[3,0],[4,0],[5,0]]))
def difference (a, b):
    return list(set(b) - set(a))
#difference([1,2,3], [1,2,3,4,5])

import itertools
from scipy.interpolate import interp1d
from functools import partial

def grouper (n, iterable):
    "grouper(3, 'ABCDEFG', 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return list(itertools.zip_longest(*args))[:-1]

def spectrum (vector):
    '''get the power spectrum of a vector of raw EEG data'''
    A = np.fft.fft(vector)
    ps = np.abs(A)*2.0
    ps = ps[:len(ps)//2]
    return ps

def binned (pspectra, n):
    '''compress an array of power spectra into vectors of length n'''
    l = len(pspectra)
    array = np.zeros([l,n])
    for i,ps in enumerate(pspectra):
        x = np.arange(1,len(ps)+1)
        f = interp1d(x,ps)#/np.sum(ps))
        array[i] = f(np.arange(1, n+1))
    index = np.argwhere(array[:,0]==-1)
    array = np.delete(array,index,0)
    return array

def feature_vector (bins, readings): # A function we apply to each group of power spectra
  '''
  Create 100, log10-spaced bins for each power spectrum.
  For more on how this particular implementation works, see:
      http://people.ischool.berkeley.edu/~chuang/pubs/MMJC15.pdf
  '''
  bins = binned(list(map(spectrum, readings)),
                bins)
  return np.log10(np.mean(bins, 0))


from functools import reduce

def features (raws, bins=50, time_resolution=250, spectra_compression=2 ):
    featureVectorF = partial(feature_vector, bins)
    spectra = map(spectrum, grouper(time_resolution, raws))
    groupedSpectra = grouper(spectra_compression, spectra)
    vs = list(map(featureVectorF, groupedSpectra))
    return np.array(vs)

#breathe = load_recording(
#    find_recordings(task='breathe')[0])

#features(breathe[1], bins=25, spectra_compression=1).shape

def featureify (acc, df, bins=bns, time_res=t_res, spectra_compress=cmprs):
    f = lambda df, n: features(df[n],
                               bins=bins,
                               time_resolution=time_res,
                               spectra_compression=spectra_compress)

    fs = [f(df, chan) for chan in channels]
    return acc + np.concatenate(fs, 1).tolist()

def vectors (filenames):
    return np.array(reduce(featureify,
                  # a list of dataframes of recordings
                  map(load_recording, filenames),
                  []))

def load_feature_vectors(id=None, task=None):
    return vectors(find_recordings(id, task))

breathe_features = load_feature_vectors(id='001', task='breathe')

# np.array(breathe_features).shape

# here's a realy simple test of authentication power
from sklearn import preprocessing

def vectors_labels (list1, list2):
    def label (l):
        return lambda x: l
    X = np.concatenate([list1, list2])
    y = np.array(list(map(label(0), list1)) + list(map(label(1), list2)))
    return X, y

# scaler  = preprocessing.RobustScaler()
# max_breathe = load_feature_vectors(id='001', task='breathe')
# john_breathe = load_feature_vectors(id='002', task='breathe')
# X, y = vectors_labels(max_breathe, john_breathe)
# X = scaler.fit_transform(X)
# cv = cross_val_svm(X,y,7)
# (cv.mean(),cv.std())

# here's a fancier version of above using xgboost
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn import metrics
#from sklearn.grid_search impo

# eval_metrics:
# http://xgboost.readthedocs.io/en/latest//parameter.html
metrics = ['auc', 'map']

def cv (alg,X,y,nfold=7):
    xgtrain = xgb.DMatrix(X,y)
    param = alg.get_xgb_params()
    cvresults = xgb.cv(param,
                      xgtrain,
                      num_boost_round=alg.get_params()['n_estimators'],
                      nfold=nfold,
                      metrics=metrics,
                      early_stopping_rounds=50)
    alg.set_params(n_estimators=cvresults.shape[0])
    alg.fit(X,y,eval_metric=metrics)
    return alg, cvresults

def inspect (alg):
    return alg.booster().get_score(importance_type='weight')
    #Predict training set:
    #dtrain_predictions = alg.predict(X)
    #dtrain_predprob = alg.predict_proba(X)[:,1]
    ##Print model report:
    #print("Accuracy : %.4g" % metrics.accuracy_score(y, dtrain_predictions))
    #print("AUC Score (Train): %f" % metrics.roc_auc_score(y, dtrain_predprob))
    #feat_imp = pd.Series(features).sort_values(ascending=False)
    # return alg.booster().get_fscore()
    #feat_imp.plot(kind='bar', title='Feature Importances')
    #plt.ylabel('Feature Importance Score')


def fresh_xgb ():
    return XGBClassifier(
         learning_rate =0.1,
         n_estimators=1000,
         max_depth=5,
         min_child_weight=1,
         gamma=0,
         subsample=0.8,
         colsample_bytree=0.8,
         objective= 'binary:logistic',
         nthread=4,
         scale_pos_weight=1,
         seed=27) 
#
#X, y = vectors_labels(
#    max_breathe,
#    john_breathe)
#
#alg, cvresults = cv(fresh_xgb(), X,y)
## see result of last cv round
##cvresults[-1:]
#inspect(alg)


Populating the interactive namespace from numpy and matplotlib


In [106]:
# returns FAR, FRR
def far_frr (alg, X, y):
    false_accept = 0
    false_reject = 0
    predicted_labels = alg.predict(X)
    for idx, predicted_label in enumerate(predicted_labels):
        correct_label = y[idx]
        if (predicted_label==correct_label):
           continue
        elif (predicted_label==0):
           false_accept+=1
           continue
        false_reject+=1
    def rate (num):
        return num/len(X)
    return { 'FAR': rate(false_accept),
             'FRR': rate(false_reject),
             'f-scores': inspect(alg), }

def test_inherence (alg, stats, right_person_wrong_task_vectors):
    labels = [0 for i in(right_person_wrong_task_vectors)]
    right_person_wrong_task_stats = far_frr(alg, right_person_wrong_task_vectors, labels)
    return right_person_wrong_task_stats

def test_auth (task, percent_to_withold=0, against_all_tasks=True):

    def apply_if_witholding (f, vectors):
        if (percent_to_withold):
            return f(percent_to_withold, vectors)
        return vectors

    def trainset (vectors):
        return apply_if_witholding(withold_final, vectors)

    def testset (vectors):
        return apply_if_witholding(get_final, vectors)

    # get `task` by this `subject`
    # HACK just using subj 1 for now
    this_persons_features =  load_feature_vectors('001', task)

    # we can test against only this task from others
    if not against_all_tasks:
        # get `task` NOT from this `subject`
        not_this_persons_features =  load_feature_vectors('002', task=task)  

    # or against all tasks from others
    elif against_all_tasks:
        this_persons_recordings = find_recordings('001')
        not_this_persons_recordings = difference(this_persons_recordings, files)
        not_this_persons_features = vectors(not_this_persons_recordings)


    # turn these pos/neg features into vectors, labels X, y
    trainX, trainy = vectors_labels(trainset(this_persons_features),
                                    trainset(not_this_persons_features))


    # fit an alg over 7 cv rounds
    alg, cvresults = cv(fresh_xgb(),
                        trainX, trainy,
                        nfold=7)
    #inspect(alg)

    # turn these pos/neg features into vectors, labels X, y
    testX, testy= vectors_labels(testset(this_persons_features),
                                 testset(not_this_persons_features))


    # get far, frr
    stats = far_frr(alg, testX, testy)

    # test right person wrong task
    right_person_right_task_recordings = find_recordings('001', task)
    right_person_wrong_task_vectors = vectors(difference(right_person_right_task_recordings,
                                                         find_recordings('001')))
    inherence_stats = test_inherence(alg, stats, right_person_wrong_task_vectors)

    # HACK
    stats['right_person_wrong_task_far'] = inherence_stats['FAR']
    stats['right_person_wrong_task_frr'] = inherence_stats['FRR']
    return stats

test_auth(task_names[2],
          against_all_tasks=True)

#alg, cvresults = cv(fresh_xgb(), X,y)
##far_frr(alg, X, y)


{'FAR': 0.0,
 'FRR': 0.0,
 'f-scores': {'f1': 1},
 'right_person_wrong_task_far': 0.0,
 'right_person_wrong_task_frr': 0.0}

In [107]:
'''
when I do 

    for task in task_names:
        print(task, test_auth(task))

the results are perfect...........too perfect
I think we are overfitting.

its confusing, but in the cv round, we are doing 7 rounds of cv to generate some parameters
then, we fit those parameters to the algorithm, and return it
so, when we test on that same data later, we are effectively testing on the train set,
since the paramaters were generated from there.


as a simple (dumb) solution, lets withold the last n samples of each group (positive, negative) for testing
'''

stats = ['FAR','FRR', 'f-scores', 'right_person_wrong_task_far', 'right_person_wrong_task_frr']
df = pd.DataFrame(columns=['task'] + stats)

for idx, task in enumerate(task_names):
    print('running', task)
    results = test_auth(task,
                        percent_to_withold=0.7)
    results_stats = [results[stat] for stat in stats]
    row = [task]+results_stats
    df.loc[idx] = row
    #df.append(row)

df

          task  FAR  FRR   f-scores  right_person_wrong_task_far  \
0       speech  0.0  0.0  {'f1': 1}                          0.0   
1         face  0.0  0.0  {'f1': 1}                          0.0   
2      breathe  0.0  0.0  {'f1': 1}                          0.0   
3  listennoise  0.0  0.0  {'f1': 1}                          0.0   
4       song_o  0.0  0.0  {'f1': 1}                          0.0   
5         song  0.0  0.0  {'f1': 1}                          0.0   
6     sequence  0.0  0.0  {'f1': 1}                          0.0   
7   listentone  0.0  0.0  {'f1': 1}                          0.0   
8    breathe_o  0.0  0.0  {'f1': 1}                          0.0   
9        sport  0.0  0.0  {'f1': 1}                          0.0   

   right_person_wrong_task_frr  
0                          0.0  
1                          0.0  
2                          0.0  
3                          0.0  
4                          0.0  
5                          0.0  
6                   

running sport


running breathe_o


running listentone


running sequence


...

running song_o


running listennoise


running breathe


running face


running speech


In [108]:
from scipy.stats import ttest_rel

wrong_task_frr = df['right_person_wrong_task_frr'].tolist()
right_task_frr = df['FRR'].tolist()
pvalue_frr = ttest_rel(right_task_frr, wrong_task_frr)[1]
#wrong_task_far = df['right_person_wrong_task_far'].tolist()
#right_task_far = df['FAR'].tolist()
#pvalue_far = ttest_rel(right_task_far, wrong_task_far)[1]
print('far diff?', pvalue_far, 'frr diff?', pvalue_frr)
#chisquare(
#    df['right_person_wrong_task_frr'].tolist(),
#    f_exp=df['FRR'].tolist())


/home/ffff/.local/lib/python3.5/site-packages/scipy/stats/stats.py:3658: RuntimeWarning: invalid value encountered in absolute
  prob = distributions.t.sf(np.abs(t), df) * 2  # use np.abs to get upper tail


far diff? nan frr diff? nan


In [109]:
df.to_csv(outFile+'-'+str(pvalue_frr)+'.csv')